# 04. 생성한 합성 데이터셋 검증

## Step 1. 통합 및 의미론적 중복 제거 (Semantic Deduplication)
### 목표: 생성된 파일에 흩어진 '같은 질문'을 찾아 하나만 남기고 삭제

### 방법:

1. 파일 병합 (Total Pool 생성).

2. 임베딩(Embedding) 기술로 질문 간의 유사도 측정(유사도가 0.85 이상인 질문들을 같은 그룹으로 묶고, 그중 답변 품질이 가장 좋은 1개만 남기고 삭제)

In [31]:
"""
## option 1
- 띄어쓰기 단위로 임베딩하는 스크립트
- 질문만 임베딩하여 유사도 계산
- 한글 코사인 유사도가 잘 측정되지 않음
"""

import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 1. 데이터 로드 및 병합 (Data Loading)
file_paths = {}

# 0.0부터 0.4까지 (range(5))
for i in range(5): 
    temp_val = i / 10
    filename = f'./data/QA/qwen3-coder-A3B-instruct/qa_dataset_temp_{temp_val:.1f}.json'
    file_paths[temp_val] = filename

all_data = []
for temp, path in file_paths.items():
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            for entry in data:
                entry['temperature'] = temp # 출처(Temp) 표시
            all_data.extend(data)
    except FileNotFoundError:
        print(f"⚠️ 경고: 파일을 찾을 수 없습니다 - {path}")

df = pd.DataFrame(all_data)

# 2. 임베딩 및 유사도 계산 (Embedding & Similarity)
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['question']) # 질문만 임베딩 진행
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 3. 중복 제거 및 선별
visited = set()
deduplicated_data = []
duplicate_groups = []  # 확인용 리스트

threshold = 0.85 

for i in range(len(df)):
    if i in visited:
        continue
    
    similar_indices = np.where(cosine_sim[i] > threshold)[0]
    cluster = df.iloc[similar_indices].to_dict('records')
    
    # 중복 그룹 저장 (클러스터 크기가 1보다 큰 경우)
    if len(cluster) > 1:
        group_info = {
            "group_id": len(duplicate_groups) + 1,
            "count": len(cluster),
            "questions": cluster
        }
        duplicate_groups.append(group_info)
    
    for idx in similar_indices:
        visited.add(idx)
    
    # 대표 질문 선정 (Temperature가 가장 낮은 것)
    best_candidate = sorted(cluster, key=lambda x: x['temperature'])[0]
    deduplicated_data.append(best_candidate)

# 4. 결과 저장
output_df = pd.DataFrame(deduplicated_data)

# 4-1. JSONL 포맷 저장 (한 줄에 하나씩)
output_df.to_json('deduplicated_dataset.jsonl', orient='records', lines=True, force_ascii=False, indent=4)

# 4-2. JSON 리스트 포맷 저장 (전체가 [ ]로 감싸짐)
output_df.to_json('deduplicated_dataset.json', orient='records', lines=False, force_ascii=False, indent=4)

# 4-3. 중복 그룹 검토용 저장
with open('duplicate_groups_check.json', 'w', encoding='utf-8') as f:
    json.dump(duplicate_groups, f, indent=4, ensure_ascii=False)

# 5. 요약 출력
print(f"전체 데이터 수: {len(df)}")
print(f"중복 제거 후: {len(output_df)}")
print(f"발견된 중복 그룹 수: {len(duplicate_groups)}")
print(f"파일 저장 완료:")
print(f"   1. deduplicated_dataset.jsonl (라인별)")
print(f"   2. deduplicated_dataset.json (리스트형)")
print(f"   3. duplicate_groups_check.json (중복그룹확인)")

전체 데이터 수: 1405
중복 제거 후: 835
발견된 중복 그룹 수: 292
파일 저장 완료:
   1. deduplicated_dataset.jsonl (라인별)
   2. deduplicated_dataset.json (리스트형)
   3. duplicate_groups_check.json (중복그룹확인)


In [ ]:
"""
## option 2
- 형태소 단위로 임베딩하는 스크립트
- 질문만 임베딩하여 유사도 계산
"""

import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt

# 1. 데이터 로드 및 병합 (Data Loading)
file_paths = {}

# 0.0부터 0.4까지 (range(5))
for i in range(5): 
    temp_val = i / 10
    filename = f'./data/QA/qwen3-coder-A3B-instruct/qa_dataset_temp_{temp_val:.1f}.json'
    file_paths[temp_val] = filename

all_data = []
for temp, path in file_paths.items():
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            for entry in data:
                entry['temperature'] = temp # 출처(Temp) 표시
            all_data.extend(data)
    except FileNotFoundError:
        print(f"⚠️ 경고: 파일을 찾을 수 없습니다 - {path}")

df = pd.DataFrame(all_data)

# 2. 임베딩 및 유사도 계산 (Embedding & Similarity)
okt = Okt() 

def korean_tokenizer(text): # 토크나이저 함수 정의 (명사, 동사, 형용사만 추출하거나 형태소 단위로 쪼갬)
    # 형태소 분석 결과에서 '형태소(form)'만 추출
    tokens = okt.morphs(text)
    return [t.lower() for t in tokens]

vectorizer = TfidfVectorizer(
    tokenizer=korean_tokenizer,
    token_pattern=None,
    use_idf=False,
    norm='l2'        
)

tfidf_matrix = vectorizer.fit_transform(df['question']) # 질문만 임베딩 진행
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 3. 중복 제거 및 선별
visited = set()
deduplicated_data = []
duplicate_groups = []  # 확인용 리스트

threshold = 0.7

for i in range(len(df)):
    if i in visited:
        continue
    
    similar_indices = np.where(cosine_sim[i] > threshold)[0]
    cluster = df.iloc[similar_indices].to_dict('records')
    
    # 중복 그룹 저장 (클러스터 크기가 1보다 큰 경우)
    if len(cluster) > 1:
        group_info = {
            "group_id": len(duplicate_groups) + 1,
            "count": len(cluster),
            "questions": cluster
        }
        duplicate_groups.append(group_info)
    
    for idx in similar_indices:
        visited.add(idx)
    
    # 대표 질문 선정 (Temperature가 가장 낮은 것)
    best_candidate = sorted(cluster, key=lambda x: x['temperature'])[0]
    deduplicated_data.append(best_candidate)

# 4. 결과 저장
output_df = pd.DataFrame(deduplicated_data)

# 4-1. JSONL 포맷 저장 (한 줄에 하나씩)
output_df.to_json('deduplicated_dataset.jsonl', orient='records', lines=True, force_ascii=False, indent=4)

# 4-2. JSON 리스트 포맷 저장 (전체가 [ ]로 감싸짐)
output_df.to_json('deduplicated_dataset.json', orient='records', lines=False, force_ascii=False, indent=4)

# 4-3. 중복 그룹 검토용 저장
with open('duplicate_groups_check.json', 'w', encoding='utf-8') as f:
    json.dump(duplicate_groups, f, indent=4, ensure_ascii=False)

# 5. 요약 출력
print(f"전체 데이터 수: {len(df)}")
print(f"중복 제거 후: {len(output_df)}")
print(f"발견된 중복 그룹 수: {len(duplicate_groups)}")
print(f"파일 저장 완료:")
print(f"   1. deduplicated_dataset.jsonl (라인별)")
print(f"   2. deduplicated_dataset.json (리스트형)")
print(f"   3. duplicate_groups_check.json (중복그룹확인)")

전체 데이터 수: 1405
중복 제거 후: 621
발견된 중복 그룹 수: 336
파일 저장 완료:
   1. deduplicated_dataset.jsonl (라인별)
   2. deduplicated_dataset.json (리스트형)
   3. duplicate_groups_check.json (중복그룹확인)


In [34]:
# 카테고리별 개수 출력

import json
from collections import Counter

# 데이터 로드
file_path = 'deduplicated_dataset.json'

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 데이터가 리스트인 경우 바로 사용하도록 수정
if isinstance(data, list):
    items = data
else:
    # 만약 딕셔너리 구조라면 기존 방식 시도
    items = data.get('fullContent', [])

# 카테고리만 추출하여 개수 세기
categories = [item.get('category') for item in items if isinstance(item, dict) and 'category' in item]
category_counts = Counter(categories)

# 결과 출력
print(f"총 데이터 개수: {len(items)}개")
print("-" * 30)
for category, count in category_counts.most_common():
    print(f"{category}: {count}개")

총 데이터 개수: 621개
------------------------------
simple: 329개
procedural: 174개
negative: 118개


In [35]:
# json 데이터 정렬
import json

# 1. 데이터 불러오기
file_path = 'deduplicated_dataset.json'

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 데이터가 리스트인지 딕셔너리인지 확인 후 리스트 추출
items = data if isinstance(data, list) else data.get('fullContent', [])

# 2. 카테고리 우선순위 정의
category_order = {
    'simple': 1,
    'procedural': 2,
    'negative': 3
}

"""
# 3. 정렬 수행 (3단계)
# 1순위: source_file (파일명 오름차순)
# 2순위: category (지정된 순서)
# 3순위: question (가나다순 - instruction 역할)
sorted_items = sorted(items, key=lambda x: (
    x.get('source_file', ''),                  # 1. 파일명
    category_order.get(x.get('category'), 99), # 2. 카테고리
    x.get('question', '')                      # 3. 질문(instruction) 가나다순
))
"""

# 2순위 제거
sorted_items = sorted(items, key=lambda x: (
    x.get('source_file', ''),                  # 1. 파일명
    x.get('question', '')                      # 3. 질문(instruction) 가나다순
))


# 4. 결과 확인 (상위 5개 출력)
print("정렬 결과 확인 (상위 5개):")
for item in sorted_items[:5]:
    print(f"File: {item.get('source_file')} | Category: {item.get('category')} | Question: {item.get('question')}")

# 5. 파일로 저장
output_path = 'sorted_dataset_v2.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(sorted_items, f, ensure_ascii=False, indent=4)

print(f"\n최종 정렬된 데이터가 '{output_path}' 파일로 저장되었습니다.")

정렬 결과 확인 (상위 5개):
File: 01.txt | Category: simple | Question: 고용계약서는 누구에게 제출해야 합니까?
File: 01.txt | Category: simple | Question: 고용계약서는 누구와 체결됩니까?
File: 01.txt | Category: simple | Question: 고용계약서는 어떤 사항들을 포함해야 합니까?
File: 01.txt | Category: simple | Question: 고용계약서는 언제부터 적용되나요?
File: 01.txt | Category: procedural | Question: 고용계약서를 작성한 후 직원이 서명해야 하나요?

최종 정렬된 데이터가 'sorted_dataset_v2.json' 파일로 저장되었습니다.
